# Hyperparameter Tuning

In [49]:
# Load Libraries
import pandas as pd
import numpy as np

# Set random seed
np.random.seed(42)

### Load Data

In [50]:
# Load data to a DataFrame
beanDF = pd.read_excel('DryBeanDataset/Dry_Bean_Dataset.xlsx')

In [51]:
# Drop least useful features
beanDF.drop(columns=['Extent', 'Solidity', 'Eccentricity', 'ShapeFactor3'], inplace=True)

## Model Selection & Evaluation
Used GridSearchCV or RandomizedSearchCV to find the best hyperparameters for each modeling type

In [52]:
# Load libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Split Training and Testing Data

In [53]:
# Set up features target sets
X = beanDF[beanDF.columns[:-1]]
y = beanDF.Class

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, 
                                                    random_state=42)

# Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test  = scaler.transform(X_test)

# Keras

## Neural Network - Keras

In [54]:
# Load libraries
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [55]:
# Encode the target variables
le = LabelEncoder()
le.fit(y)

le_y_train = le.transform(y_train)
le_y_test  = le.transform(y_test)

from keras.utils.np_utils import to_categorical
cat_y_train = to_categorical(le_y_train)
cat_y_test  = to_categorical(le_y_test)

# NOTE: Add dropout? (see M5_test2) 

In [10]:
# Set the number of features
number_of_features = X.shape[1]

# Start neural network
network = Sequential()

# Add fully connected layer w/a ReLU activation function
network.add(Dense(units=100, activation='relu', 
                  input_shape=(number_of_features,)))

# Add fully connected layer w/a ReLU activation function
network.add(Dense(units=100, activation='relu'))

# Add fully connected layer w/a softmax activation function
network.add(Dense(units=7, activation='softmax'))

# Compile neural network
network.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])

In [11]:
# Fit model
history = network.fit(X_train, cat_y_train, 
                      epochs=3, 
                      batch_size=100, 
                      validation_data=(X_test, cat_y_test))

Wall time: 0 ns
Epoch 1/3
96/96 [==============================] - 4s 19ms/step - loss: 1.2479 - accuracy: 0.5839 - val_loss: 0.3482 - val_accuracy: 0.9011
Epoch 2/3
96/96 [==============================] - 1s 9ms/step - loss: 0.3022 - accuracy: 0.9066 - val_loss: 0.2338 - val_accuracy: 0.9150
Epoch 3/3
96/96 [==============================] - 1s 9ms/step - loss: 0.2265 - accuracy: 0.9202 - val_loss: 0.2283 - val_accuracy: 0.9145


In [12]:
round(history.history['accuracy'][np.argmin(history.history['loss'])]*100, 2)

91.94

In [14]:
# Fit model
history = network.fit(X_train, cat_y_train, 
                      epochs=15, 
                      batch_size=5, 
                      validation_data=(X_test, cat_y_test))

Wall time: 0 ns
Epoch 1/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.2328 - accuracy: 0.9173 - val_loss: 0.2133 - val_accuracy: 0.9263
Epoch 2/15
1906/1906 [==============================] - 7s 4ms/step - loss: 0.2181 - accuracy: 0.9188 - val_loss: 0.1965 - val_accuracy: 0.9283
Epoch 3/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.2121 - accuracy: 0.9212 - val_loss: 0.2103 - val_accuracy: 0.9234
Epoch 4/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.2061 - accuracy: 0.9206 - val_loss: 0.2086 - val_accuracy: 0.9231
Epoch 5/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.2030 - accuracy: 0.9252 - val_loss: 0.2196 - val_accuracy: 0.9219
Epoch 6/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.1999 - accuracy: 0.9258 - val_loss: 0.1993 - val_accuracy: 0.9283
Epoch 7/15
1906/1906 [==============================] - 6s 3ms/step - loss: 0.2023 - accuracy: 0.9248 - val_loss: 0.2069

In [15]:
round(history.history['accuracy'][np.argmin(history.history['loss'])]*100, 2)

92.35

In [16]:
history.history['accuracy']

[0.9172877073287964,
 0.9187572002410889,
 0.9211714267730713,
 0.920646607875824,
 0.9251600503921509,
 0.9257898330688477,
 0.9248451590538025,
 0.9239004850387573,
 0.9234806299209595,
 0.9245302677154541,
 0.9261047840118408,
 0.9271544218063354,
 0.9234806299209595,
 0.9274693131446838,
 0.9269444942474365]

## 20.13 Tuning Neural Networks

https://www.pyimagesearch.com/2021/05/31/hyperparameter-tuning-for-deep-learning-with-scikit-learn-keras-and-tensorflow/

In [38]:
# Load libraries
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Set random seed
np.random.seed(42)

In [56]:
# Number of features
number_of_features = X.shape[1]

# Create function returning a compiled network
def create_network(optimizer='rmsprop', hiddenLayerOne=48, hiddenLayerTwo=24, 
                   initDrop=0.2, hiddenDrop=0.5, learnRate=0.01):
    
    # Start neural network
    network = Sequential()
    
    # Add a dropout layer for input layer
    network.add(Dropout(initDrop, input_shape=(number_of_features,)))
    
    # Add fully connected layer w/a ReLU activation function
    network.add(Dense(units=hiddenLayerOne, activation='relu'))#,
#                       input_shape=(number_of_features,)))
    
    # Add a dropout layer for previous hidden layer
    network.add(Dropout(hiddenDrop))

    # Add fully connected layer w/a ReLU activation function
    network.add(Dense(units=hiddenLayerTwo, activation='relu'))
                
    # Add a dropout layer for previous hidden layer
    network.add(Dropout(hiddenDrop))

    # Add fully connected layer w/a softmax activation function
    network.add(Dense(units=7, activation='softmax'))

    # Compile neural network
    network.compile(loss='categorical_crossentropy', 
                    optimizer=Adam(learning_rate=learnRate),
                    metrics=['accuracy'])

    # Return compiled network
    return network

In [66]:
# Wrap Keras model so it can be used by scikit-learn
keras = KerasClassifier(build_fn=create_network, verbose=0)

# Create hyperparameter space and initialize random search
parameter_space = {
    'epochs': [5, 10, 15],
    'batch_size': [4, 8, 16, 32],
    'hiddenLayerOne': [48, 96, 120],
    'hiddenLayerTwo': [24, 48, 96],
    'initDrop': [0.1, 0.2],
    'hiddenDrop': [0.3, 0.4, 0.5], 
    'learnRate': [1e-4, 1e-3, 1e-2],
}
rand = RandomizedSearchCV(keras, parameter_space, random_state=42, n_iter=50, 
                          scoring='accuracy', n_jobs=-1, cv=3)

In [67]:
%%time
# Fit models
rand_result = rand.fit(X_train, cat_y_train, 
                       validation_data=(X_test, cat_y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Wall time: 13min 5s


In [68]:
# Summarize search information
bestScore = rand_result.best_score_
bestParams = rand_result.best_params_
print(f"Best score is {round(bestScore*100, 2)} using {bestParams}")

Best score is nan using {'learnRate': 0.01, 'initDrop': 0.1, 'hiddenLayerTwo': 48, 'hiddenLayerOne': 120, 'hiddenDrop': 0.5, 'epochs': 5, 'batch_size': 32}


In [69]:
# Get a baseline model
model = create_network()
baseline = model.fit(X_train, cat_y_train, 
              validation_data=(X_test, cat_y_test), 
              batch_size=8, 
              epochs=10)

Epoch 1/10
1191/1191 [==============================] - 3s 2ms/step - loss: 0.9970 - accuracy: 0.6356 - val_loss: 0.3439 - val_accuracy: 0.9084
Epoch 2/10
1191/1191 [==============================] - 2s 2ms/step - loss: 0.7494 - accuracy: 0.7410 - val_loss: 0.3447 - val_accuracy: 0.8918
Epoch 3/10
1191/1191 [==============================] - 2s 2ms/step - loss: 0.7308 - accuracy: 0.7573 - val_loss: 0.3286 - val_accuracy: 0.8896
Epoch 4/10
1191/1191 [==============================] - 2s 2ms/step - loss: 0.6844 - accuracy: 0.7626 - val_loss: 0.2969 - val_accuracy: 0.9165
Epoch 5/10
1191/1191 [==============================] - 2s 2ms/step - loss: 0.6719 - accuracy: 0.7787 - val_loss: 0.3662 - val_accuracy: 0.8768
Epoch 6/10
1191/1191 [==============================] - 3s 2ms/step - loss: 0.6891 - accuracy: 0.7707 - val_loss: 0.2931 - val_accuracy: 0.9099
Epoch 7/10
1191/1191 [==============================] - 2s 2ms/step - loss: 0.7058 - accuracy: 0.7755 - val_loss: 0.3521 - val_accuracy:

In [72]:
baseline.score(X_test, cat_y_test)

AttributeError: 'History' object has no attribute 'score'

In [73]:
kerasAcc = baseline.history['accuracy'][np.argmin(baseline.history['loss'])]*100
round(kerasAcc, 2)

78.14

In [78]:
rand_result.best_score_

nan

In [80]:
# Show best parameters
print('Best parameters found:\n', rand_result.best_params_, '\n')
# Get accuracy scores
baseScore = round(baseline.history['accuracy'][np.argmin(baseline.history['loss'])]*100, 2)
score = round(rand_result.best_score_*100, 2)
print(f"Baseline Accuracy:\t{baseScore}")
print(f"Tuned Accuracy:\t\t{score}")

Best parameters found:
 {'learnRate': 0.01, 'initDrop': 0.1, 'hiddenLayerTwo': 48, 'hiddenLayerOne': 120, 'hiddenDrop': 0.5, 'epochs': 5, 'batch_size': 32} 

Baseline Accuracy:	78.14
Tuned Accuracy:		nan


In [64]:
# Summarize search information
bestScore = rand_result.best_score_
bestParams = rand_result.best_params_
print(f"Best score is {round(bestScore*100, 2)} using {bestParams}")

Best score is nan using {'learnRate': 0.01, 'initDrop': 0.1, 'hiddenLayerTwo': 48, 'hiddenLayerOne': 120, 'hiddenDrop': 0.5, 'epochs': 5, 'batch_size': 32}


In [45]:
# Summarize search information
bestScore = rand_result.best_score_
bestParams = rand_result.best_params_
print(f"Best score is {round(bestScore*100, 2)} using {bestParams}")

Best score is nan using {'learnRate': 0.001, 'initDrop': 0.2, 'hiddenLayerTwo': 48, 'hiddenLayerOne': 120, 'hiddenDrop': 0.5, 'epochs': 5, 'batch_size': 16}


In [65]:
# Initialize model with default values
model = create_network( 
              learnRate=0.01, 
              initDrop=0.1, 
              hiddenLayerOne=120, 
              hiddenLayerTwo=48, 
              hiddenDrop=0.5)

# Train network 
H = model.fit(X_train, cat_y_train,
              validation_data=(X_test, cat_y_test), 
              batch_size=32, 
              epochs=5)

accuracy = model.evaluate(X_test, cat_y_test)[1]
print(f"\nAccuracy:  {round(accuracy*100, 2)}")

Epoch 1/5
298/298 [==============================] - 1s 3ms/step - loss: 0.8064 - accuracy: 0.7130 - val_loss: 0.2487 - val_accuracy: 0.9067
Epoch 2/5
298/298 [==============================] - 1s 2ms/step - loss: 0.4773 - accuracy: 0.8430 - val_loss: 0.2640 - val_accuracy: 0.9065
Epoch 3/5
298/298 [==============================] - 1s 2ms/step - loss: 0.4718 - accuracy: 0.8532 - val_loss: 0.2282 - val_accuracy: 0.9172
Epoch 4/5
298/298 [==============================] - 1s 3ms/step - loss: 0.4702 - accuracy: 0.8426 - val_loss: 0.2329 - val_accuracy: 0.9219
Epoch 5/5
128/128 [==============================] - 0s 1ms/step - loss: 0.2175 - accuracy: 0.9209
Accuracy:  92.09


In [46]:
# Initialize model with default values
model = create_network( 
              learnRate=0.001, 
              initDrop=0.2, 
              hiddenLayerOne=120, 
              hiddenLayerTwo=48, 
              hiddenDrop=0.5)

# Train network 
H = model.fit(X_train, cat_y_train,
              validation_data=(X_test, cat_y_test), 
              batch_size=16, 
              epochs=5)

accuracy = model.evaluate(X_test, cat_y_test)[1]
print(f"\nAccuracy:  {round(accuracy*100, 2)}")

Epoch 1/5
596/596 [==============================] - 2s 3ms/step - loss: 1.2197 - accuracy: 0.5343 - val_loss: 0.3707 - val_accuracy: 0.8940
Epoch 2/5
596/596 [==============================] - 1s 2ms/step - loss: 0.6344 - accuracy: 0.7780 - val_loss: 0.2738 - val_accuracy: 0.9099
Epoch 3/5
596/596 [==============================] - 1s 2ms/step - loss: 0.5136 - accuracy: 0.8203 - val_loss: 0.2419 - val_accuracy: 0.9209
Epoch 4/5
596/596 [==============================] - 1s 2ms/step - loss: 0.4936 - accuracy: 0.8334 - val_loss: 0.2384 - val_accuracy: 0.9187
Epoch 5/5
128/128 [==============================] - 0s 2ms/step - loss: 0.2308 - accuracy: 0.9190
Accuracy:  91.9


In [37]:
# Initialize model with default values
model = create_network()

# Train network 
H = model.fit(X_train, cat_y_train, 
              validation_data=(X_test, cat_y_test), 
              batch_size=8, 
              epochs=20)

accuracy = model.evaluate(X_test, cat_y_test)[1]
print(f"\nAccuracy:  {round(accuracy*100, 2)}")

Epoch 1/20
1191/1191 [==============================] - 340s 282ms/step - loss: 1.5107 - accuracy: 0.4220 - val_loss: 0.4633 - val_accuracy: 0.8852
Epoch 2/20
1191/1191 [==============================] - 204s 171ms/step - loss: 0.8316 - accuracy: 0.7072 - val_loss: 0.3450 - val_accuracy: 0.9035
Epoch 3/20
1191/1191 [==============================] - 5s 4ms/step - loss: 0.7351 - accuracy: 0.7477 - val_loss: 0.3005 - val_accuracy: 0.9165
Epoch 4/20
1191/1191 [==============================] - 11s 9ms/step - loss: 0.6707 - accuracy: 0.7694 - val_loss: 0.2686 - val_accuracy: 0.9231
Epoch 5/20
1191/1191 [==============================] - 5s 4ms/step - loss: 0.6305 - accuracy: 0.7856 - val_loss: 0.2677 - val_accuracy: 0.9165
Epoch 6/20
1191/1191 [==============================] - 4s 3ms/step - loss: 0.6244 - accuracy: 0.7916 - val_loss: 0.2619 - val_accuracy: 0.9214
Epoch 7/20
1191/1191 [==============================] - 4s 3ms/step - loss: 0.6277 - accuracy: 0.7967 - val_loss: 0.2596 - val_

In [28]:
alpha,hidden_dim,dropout_percent,do_dropout = (0.5,4,0.2,True)

In [29]:
np.random.binomial([np.ones((len(X),hidden_dim))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

array([[1.25, 0.  , 1.25, 1.25],
       [1.25, 1.25, 1.25, 0.  ],
       [1.25, 1.25, 1.25, 0.  ],
       ...,
       [1.25, 0.  , 0.  , 1.25],
       [1.25, 1.25, 1.25, 1.25],
       [1.25, 0.  , 0.  , 1.25]])

In [57]:
# Create lists of models and accuracy scores
modelName = []
score = []
for key in models_dict:
    modelName += [key]
    model = models_dict[key]
    value = model.score(X_test, y_test)*100
    score += [round(value, 2)]

# Add Keras accuracy
modelName += ['Keras']
kerasAcc = history.history['val_accuracy'][np.argmin(history.history['loss'])]*100
score += [round(kerasAcc, 2)]

# Create DataFrame of results
d = {'Model': modelName, 'Accuracy': score}
results = pd.DataFrame(d).sort_values(by=['Accuracy'], ascending=False)
results

,Model,Accuracy
7,MLP,92.92
5,SVC,92.90
8,Keras,92.61
4,LinearSVC,92.38
1,RandomForest,92.19
0,Logistic,92.14
6,GaussianNB,89.99
2,DecisionTree,89.03
3,AdaBoost,86.29
